Executar MANUALMENTE os passos abaixo:
1. Executar a transação ZL1PM-COCKPIT
2. Selecionar o Layout OLEO
3. Executar a pesquisa (F8)
4. Exportar o resultado para a pasta C:\Users\Z0038362\OneDrive - ZF Friedrichshafen AG\Documents\Python Scripts
5. Salvar com o nome ordensAbertas
6. Clicar em Permitir nas mensagens que forem abrindo
7. Criar a coluna Resp e preencher a matricula do responsável pela troca de óleo
8. Salvar e fechar o Excel Aberto

In [1]:
### Importar bibliotecas necessárias ###
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

import time
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import pyautogui as pag

In [2]:
### Carregar a planilha que foi extraída do Excel com os números das notas
planilhaDf_ordens = pd.read_excel('ordensAbertas.xlsx')

In [3]:
### Avaliar se leu o arquivo corretamente
planilhaDf_ordens.head()

,Data da nota,Sala,Ordem,Nota,Equipamento,Descrição,Resp
0,28.02.2022,J020,321001330998,2688717,159698,Abastecimento de Oleo GC,9014
1,28.02.2022,F030,321001330999,2688718,159612,Abastecimento de Oleo GC,9013
2,28.02.2022,F040,321001331140,2688719,159613,Abastecimento de Oleo GC,9013
3,28.02.2022,B010,321001331141,2688720,159494,Abastecimento de Oleo GC,9013
4,28.02.2022,F010,321001331142,2688721,159608,Abastecimento de Oleo GC,9013


In [4]:
### Processo de Login com SAP WebGUI ###

## Abre janela para digitar usuário e senha
usuario = pag.prompt('Digita seu usuário')
senha = pag.prompt('Digita sua senha')

## criar o navegador com o webdriver do Chrome
navegador = webdriver.Chrome()

## Link de acesso do WebGUI do SAP da ZF
navegador.get("https://swdlp1.zf-world.com/sap/bc/gui/sap/its/webgui")

## Preencher as informações de Login e senha
navegador.find_element(By.XPATH, '//*[@id="sap-user"]').send_keys(usuario)
navegador.find_element(By.XPATH, '//*[@id="sap-password"]').send_keys(usuario)

## Clicar no botão para realizar o LOGIN
navegador.find_element(By.XPATH, '//*[@id="LOGON_BUTTON"]').click()

In [5]:
time.sleep(10)
navegador.switch_to.frame("ITSFRAME1");
navegador.find_element(By.XPATH, '//*[@title="Entrar código de transação"]').send_keys("ZILRUECK")
navegador.find_element(By.XPATH, '//*[@title="Entrar código de transação"]').send_keys(Keys.ENTER)

In [6]:
pos = 0
tamanhoPlaniha = len(planilhaDf_ordens)
wait = WebDriverWait(navegador, 10)

for i in planilhaDf_ordens.Ordem:
    time.sleep(4)
    ## Aguardar a tela carregar e digitar o numero da ordem que será encerrada
    fastrack = WebDriverWait(navegador, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="M0:46:::1:17"]')))
    fastrack.clear()
    navegador.find_element(By.XPATH, '//*[@id="M0:46:::1:17"]').send_keys(str(int(planilhaDf_ordens['Ordem'][pos])))
    navegador.find_element(By.XPATH, '//*[@id="M0:46:::1:17"]').send_keys(Keys.ENTER)
    time.sleep(1)
    
    ## Aguarda a tela carregar e digitar o tempo da execução da atividade, o manutentor e o texto que reabasteceu oleo
    fastrack2 = WebDriverWait(navegador, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="M0:46:1::4:17"]')))
    fastrack2.send_keys('0,5')
    navegador.find_element(By.XPATH, '//*[@id="M0:46:1::5:48"]').send_keys(str(planilhaDf_ordens['Resp'][pos]))
    navegador.find_element(By.XPATH, '//*[@id="M0:46:1::6:17"]').send_keys('Feito reabastecimento de oleo')
    navegador.find_element(By.XPATH, '//*[@id="M0:46:1::6:17"]').send_keys(Keys.ENTER)
    time.sleep(3)    

    ## Clicar no botão M
    navegador.find_element(By.XPATH, '//*[@title="Save (Ctrl+S)"]').click()
    time.sleep(3)    
   
    pos = pos + 1
    if pos > tamanhoPlaniha:
        pag.alert("Finalizado abertura das notas!")